<a href="https://colab.research.google.com/github/yumi-jun/Data_Analysis/blob/main/CNN_cat_and_dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

라이브러리 적용

In [4]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
file_path = '/content/drive/MyDrive/deepLearing/dataset'


## Part1 : 데이터 전처리


훈련 데이터

In [6]:
# 이미지 전처리 -> 가장 먼저 훈련 세트의 모든 이미지를 변형
# 과적합을 방지 하기 위해

# train_datagen : 훈련 세트의 이미지에 모든 변화를 적용할 도구를 나타내는 변수
train_datagen = ImageDataGenerator(
        rescale=1./255, # 특징 스케일링, 픽셀의 값을 255로 나누어 작용, 각 픽셀은 0~255 로 적용,
        # 모든 픽셀 값이 0과 1 사이로 나오게 됨.
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

 #이미지 증강 도구를 이미지에 적용
training_set=train_datagen.flow_from_directory(
        '/content/drive/MyDrive/deepLearing/dataset/training_set', # 훈련 세트로 가는 경로
        target_size=(64,64), # 이미지의 최종 크
        batch_size=32, # 각 배치에 이미지를 몇 개 배치할 것인
        class_mode='binary') # 결과값, 이진 모드, 카테고리 모드




Found 1755 images belonging to 1 classes.


테스트 데이터 test data

In [7]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/deepLearing/dataset/test_set',
        target_size=(64,64),
        batch_size=32,
        class_mode='binary')


Found 2 images belonging to 1 classes.


## 2. CNN  만들기


CNN 만들기


In [8]:
cnn=tf.keras.models.Sequential()


Step 1 - Convolution

In [9]:
#  filters : 처리할 이미지의 수, kernel_size : 특징 감지기의 수 : 3*3 , 정류화 활성화 함수를 사용, 이미지의 입력 형태 : 64 사이즈의 rgb 색상 :3
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu',input_shape=[64,64,3]))

Step 2 - Pooling

In [10]:
# 풀 크기 : 2 -> 추천 사이즈임
# 간격 : strides
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

Adding a second convolutional layer
: 두번째 컨볼루션 레이어 추가

왜 두번째 컨볼루션 레이어를 추가하는지..?

In [11]:
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

## Step 3 : 펼치기

In [12]:
cnn.add(tf.keras.layers.Flatten())

## Step 4 : 전결합층 연

In [13]:
# 유닛이 128개,
cnn.add(tf.keras.layers.Dense(units=128,activation='relu'))

## Step 5 : Output Layer

In [14]:
# 유닛이 1개
cnn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

## Training the CNN : CNN 훈련

In [15]:
cnn.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [16]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)


Epoch 1/25
55/55 [==============================] - 27s 440ms/step - loss: 0.0158 - accuracy: 0.9823 - val_loss: 2.4070e-32 - val_accuracy: 1.0000
Epoch 2/25
55/55 [==============================] - 26s 477ms/step - loss: 7.7399e-13 - accuracy: 1.0000 - val_loss: 1.2117e-32 - val_accuracy: 1.0000
Epoch 3/25
55/55 [==============================] - 25s 450ms/step - loss: 1.0664e-12 - accuracy: 1.0000 - val_loss: 1.2085e-32 - val_accuracy: 1.0000
Epoch 4/25
55/55 [==============================] - 22s 401ms/step - loss: 1.1654e-12 - accuracy: 1.0000 - val_loss: 1.2085e-32 - val_accuracy: 1.0000
Epoch 5/25
55/55 [==============================] - 24s 434ms/step - loss: 5.2280e-13 - accuracy: 1.0000 - val_loss: 1.2085e-32 - val_accuracy: 1.0000
Epoch 6/25
55/55 [==============================] - 22s 404ms/step - loss: 2.8459e-12 - accuracy: 1.0000 - val_loss: 1.2085e-32 - val_accuracy: 1.0000
Epoch 7/25
55/55 [==============================] - 24s 430ms/step - loss: 2.9244e-12 - accuracy: 

## 4: Making a single prediction


In [24]:
import numpy as np
from keras.preprocessing import image
test_image=image.load_img('/content/drive/MyDrive/deepLearing/dataset/test_set/single_prediction/cat_or_dog_2.jpg',target_size=(64,64))

# 예측 메소드에서는 , 특정한 입력 방식이 요구된다.
# 이미지 데이터 값을 넘파 배열 형태로 변환할 것이다.
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
result=cnn.predict(test_image)
training_set.class_indices


1/1 [==============================] - 0s 29ms/step


{'dogs': 0}